In [319]:
# Today Goal: My 1st Champion is 'Velkoz', what AD Carry is Best Duo with me?
import pandas as pd
import sys
import os
import requests
import time

In [320]:
# Today api_key
api_key = 'RGAPI-9fe5f863-11c2-4a2b-aded-5d5fb62fd210'

# My summoner Name
name = '얄LU'

In [321]:
# SUMMONER-V4 API URL
requesturl = 'https://kr.api.riotgames.com/lol/summoner/v4/summoners/by-name/'+name+\
'?api_key='+api_key

In [322]:
# My puuid입니다.
summoner_puuid = requests.get(requesturl).json()['puuid']

In [323]:
# matches
game_list = []
i=0
while True:
  requesturl = 'https://asia.api.riotgames.com/lol/match/v5/matches/by-puuid/'+summoner_puuid+\
'/ids?queue=450&start='+str(i)+'&count=100&api_key='+api_key #queue?

  while True:
    response = requests.get(requesturl)
    if response.ok:
        game_list_new = response.json()
        break
  
  game_list.extend(game_list_new) #append!

  if len(game_list_new) != 100:
      break

  i+=100
  time.sleep(1)

In [324]:
# 솔랭 게임 각각에 대해서 API 호출 및 row binding
games_df = pd.DataFrame()
for matchId in game_list:
    requesturl = 'https://asia.api.riotgames.com/lol/match/v5/matches/'+matchId+\
    '?api_key='+api_key
    
    while True:
        response = requests.get(requesturl)
        if response.ok:
            game_json = response.json()
            break
    
    game_df = pd.DataFrame(game_json['info']['participants'])[['puuid','championId','win','kills','deaths','assists']]
    game_df = game_df[game_df.puuid == summoner_puuid][['championId','win','kills','deaths','assists']]
        
    games_df = pd.concat([games_df,game_df])
    time.sleep(1)

In [333]:
games_df = games_df.groupby(['championId'],as_index=False).\
apply(lambda x: pd.Series({'win_count':sum(x.win),
                           'count':len(x),
                           'kills':sum(x.kills),
                           'deaths':sum(x.deaths),
                           'assists':sum(x.assists)
                          }))

In [334]:
# 챔피언 Key값에 대한 Value 가져오기
last_version = requests.get("https://ddragon.leagueoflegends.com/api/versions.json").json()[0]
champion_constant = requests.get("http://ddragon.leagueoflegends.com/cdn/"+last_version+"/data/ko_KR/champion.json")

In [335]:
# Json 파일을 DataFrame으로 변환
champion_df = pd.DataFrame(champion_constant.json()['data']).T[['key','name']]

# 변수형 문자 -> 숫자
champion_df['key'] = pd.to_numeric(champion_df['key'])

In [336]:
# 확인할 변수
COLUMNS = ['championName','win','kills','deaths','assists','lane']

In [337]:
games_df = games_df.merge(champion_df,
                          how = 'left',
                          left_on = 'championId',
                          right_on = 'key'
                         )[['name','count','win_count','kills','deaths','assists']]

In [338]:
games_df['WR'] = games_df['win_count'] / games_df['count']
games_df['KDA'] = (games_df['kills']+games_df['assists']) / games_df['deaths']

In [339]:
games_df.sort_values('count',ascending = False)[['name','WR','count','KDA']]

,name,WR,count,KDA
36,브라움,0.833333,6,4.363636
9,모르가나,0.666667,6,4.642857
37,징크스,0.800000,5,4.416667
35,벨코즈,0.600000,5,6.076923
23,아리,0.000000,4,2.593750
17,나서스,0.333333,3,3.450000
11,애니비아,0.333333,3,3.538462
10,트위치,0.333333,3,2.894737
13,카르마,0.000000,3,3.218750
27,직스,0.666667,3,3.565217
